In [12]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, BatchNormalization, Activation
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint
from tensorflow.keras.applications import VGG16
from sklearn.metrics import classification_report
from time import time
from pathlib import Path

In [13]:
print("TensorFlow version:", tf.__version__)
print("Available devices:")
for device in tf.config.list_physical_devices():
    print(device)

TensorFlow version: 2.19.0
Available devices:
PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')


In [14]:
data_path = './fer2013'
batch_size = 64

In [15]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sn
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, BatchNormalization, Activation
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint
from tensorflow.keras.applications import VGG16
from sklearn.metrics import classification_report
from time import time

train_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,
    rotation_range=5,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode='nearest'
)

valid_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)
test_datagen = ImageDataGenerator(rescale=1./255)

train_dataset = train_datagen.flow_from_directory(
    directory=data_path + '/train',
    target_size=(48, 48),
    color_mode='rgb',
    class_mode='categorical',
    subset='training',
    batch_size=batch_size
)

valid_dataset = valid_datagen.flow_from_directory(
    directory=data_path + '/train',
    target_size=(48, 48),
    color_mode='rgb',
    class_mode='categorical',
    subset='validation',
    batch_size=batch_size
)

test_dataset = test_datagen.flow_from_directory(
    directory=data_path + '/test',
    target_size=(48, 48),
    color_mode='rgb',
    class_mode='categorical',
    batch_size=batch_size,
    shuffle=False
)

Found 22968 images belonging to 7 classes.
Found 5741 images belonging to 7 classes.
Found 7178 images belonging to 7 classes.


In [16]:
base_model = VGG16(input_shape=(48, 48, 3), include_top=False, weights="imagenet")
for layer in base_model.layers[:-4]:
    layer.trainable = False

model = Sequential([
    base_model,
    Dropout(0.5),
    Flatten(),
    BatchNormalization(),
    Dense(32, kernel_initializer='he_uniform'),
    BatchNormalization(),
    Activation('relu'),
    Dropout(0.5),
    Dense(32, kernel_initializer='he_uniform'),
    BatchNormalization(),
    Activation('relu'),
    Dropout(0.5),
    Dense(32, kernel_initializer='he_uniform'),
    BatchNormalization(),
    Activation('relu'),
    Dense(7, activation='softmax')
])

model.summary()

58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 102s 2us/step


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ vgg16 (Functional)              │ (None, 1, 1, 512)      │    14,714,688 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 1, 1, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 512)            │         2,048 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 32)             │        16,416 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 32)             │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation (Activation)         │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         1,056 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 32)             │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_1 (Activation)       │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 32)             │         1,056 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 32)             │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_2 (Activation)       │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 7)              │           231 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 14,735,879 (56.21 MB)

 Trainable params: 7,099,399 (27.08 MB)

 Non-trainable params: 7,636,480 (29.13 MB)

In [20]:
from tensorflow.keras import backend as K

def f1_score(y_true, y_pred):
    y_pred_bin = K.round(y_pred)
    tp = K.sum(K.cast(y_true * y_pred_bin, 'float'), axis=0)
    predicted_positives = K.sum(K.cast(y_pred_bin, 'float'), axis=0)
    possible_positives = K.sum(K.cast(y_true, 'float'), axis=0)
    precision = tp / (predicted_positives + K.epsilon())
    recall = tp / (possible_positives + K.epsilon())
    f1 = 2 * precision * recall / (precision + recall + K.epsilon())
    return K.mean(f1)

METRICS = [
    tf.keras.metrics.CategoricalAccuracy(name='accuracy'),
    tf.keras.metrics.Precision(name='precision'),
    tf.keras.metrics.Recall(name='recall'),
    tf.keras.metrics.AUC(name='auc'),
    f1_score
]

lrd = ReduceLROnPlateau(monitor='val_loss', patience=10, verbose=1, factor=0.5, min_lr=1e-6)
es = EarlyStopping(patience=20, restore_best_weights=True, verbose=1)
mcp = ModelCheckpoint('results/best_model.keras', save_best_only=True)

In [21]:
class PerClassMetricsCallback(tf.keras.callbacks.Callback):
    def __init__(self, val_dataset, class_names):
        super().__init__()
        self.val_dataset = val_dataset
        self.class_names = class_names

    def on_epoch_end(self, epoch, logs=None):
        y_true = []
        y_pred = []
        for batch_x, batch_y in self.val_dataset:
            preds = self.model.predict(batch_x, verbose=0)
            y_true.extend(np.argmax(batch_y, axis=1))
            y_pred.extend(np.argmax(preds, axis=1))
            if len(y_true) >= self.val_dataset.samples:
                break
        report = classification_report(y_true, y_pred, target_names=self.class_names, zero_division=0)
        print(f"\n[Epoch {epoch+1}] Classification Report:\n{report}")

class_names = list(train_dataset.class_indices.keys())
per_class_cb = PerClassMetricsCallback(valid_dataset, class_names)

In [22]:
# Train
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=METRICS)

start_time = time()

history = model.fit(
    train_dataset,
    validation_data=valid_dataset,
    epochs=5,
    callbacks=[lrd, es, mcp, per_class_cb],
    verbose=1
)

end_time = time()
training_time = end_time - start_time
print(f"Training time: {training_time:.2f} seconds")

Epoch 1/5
359/359 ━━━━━━━━━━━━━━━━━━━━ 0s 544ms/step - accuracy: 0.2560 - auc: 0.6623 - f1_score: 0.0150 - loss: 1.7991 - precision: 0.4835 - recall: 0.0072
[Epoch 1] Classification Report:
              precision    recall  f1-score   support

       angry       0.00      0.00      0.00       799
     disgust       0.00      0.00      0.00        87
        fear       0.00      0.00      0.00       819
       happy       0.26      1.00      0.41      1443
     neutral       0.00      0.00      0.00       993
         sad       0.00      0.00      0.00       966
    surprise       0.78      0.11      0.19       634

    accuracy                           0.26      5741
   macro avg       0.15      0.16      0.08      5741
weighted avg       0.15      0.26      0.12      5741

359/359 ━━━━━━━━━━━━━━━━━━━━ 279s 774ms/step - accuracy: 0.2560 - auc: 0.6623 - f1_score: 0.0150 - loss: 1.7990 - precision: 0.4836 - recall: 0.0073 - val_accuracy: 0.2630 - val_auc: 0.6628 - val_f1_score: 0.0023 

KeyboardInterrupt: 

In [ ]:
# Plot results
def plot_history(history):
    metrics = ['accuracy', 'loss', 'auc', 'precision', 'f1_score']
    fig, axes = plt.subplots(1, len(metrics), figsize=(25, 5))
    for i, m in enumerate(metrics):
        axes[i].plot(history.history[m])
        axes[i].plot(history.history[f'val_{m}'])
        axes[i].set_title(m)
        axes[i].legend(['train', 'val'])
    plt.show()

plot_history(history)